 <h1 style="text-align: center">Automacao com funcao</h1>
 

In [72]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [73]:
def on_bollinger(ticker, period, dispersion, start, end, target):
    # Criando Variaveis
    dataframe = pdr.get_data_yahoo(ticker, start, end)
    dataframe["Desvio"] = dataframe["Adj Close"].rolling(period).std()
    dataframe["MM"] = dataframe["Adj Close"].rolling(period).mean()
    dataframe["Banda_Superior"] = dataframe["MM"] + (dataframe["Desvio"] * dispersion)
    dataframe["Banda_Inferior"] = dataframe["MM"] - (dataframe["Desvio"] * dispersion)
    
    dataframe = dataframe.dropna(axis = 0) # Removendo registros zerados
    
    # Construcao de Alvos
    dataframe.loc[:, "Retorno"] = dataframe["Adj Close"].pct_change(target)
    dataframe.loc[:, "Alvo"] = dataframe["Retorno"].shift(- target)
    
    dataframe = dataframe.dropna(axis = 0)
    
    # Regra de trade
    dataframe.loc[:, "Regra"] = np.where(dataframe.loc[:, "Adj Close"] > dataframe.loc[:, "Banda_Superior"], 1, 0)
    dataframe.loc[:, "Regra"] = np.where(dataframe.loc[:, "Adj Close"] < dataframe.loc[:, "Banda_Inferior"], - 1, dataframe.loc[:, "Regra"])
    
    # Aplicando Regra no Alvo
    dataframe.loc[:, "Trade"] = dataframe.loc[:, "Regra"] * dataframe.loc[:, "Alvo"]
    
    # Resultado acumulado por juros simples
    dataframe.loc[:, "Retorno_Trade"] = dataframe["Trade"].cumsum()
    
    return(dataframe.iloc[-1][-1])

In [74]:
# Definindo parametros da funcao

ticker = "^BVSP"
start = "2015-01-01"
end = "2022-02-11"
period = 50
dispersion = 2
target = 5

In [75]:
on_bollinger(ticker, period, dispersion, start, end, target)

[*********************100%***********************]  1 of 1 completed


2.478564119115134

In [76]:
def plot_resultado(data_frame):
    data_frame["Retorno_Trade"].plot(figsize=(20, 15), linewidth = 3, fontsize = 15, color = "green")
    
    plt.xlabel("Data", fontsize = 18)
    plt.ylabel("Pontos", fontsize = 18)
    plt.title("Resultado", fontsize = 26)
    plt.legend()

In [77]:
data_bollinger = on_bollinger(ticker, period, dispersion, start, end, target)

[*********************100%***********************]  1 of 1 completed


In [78]:
data_bollinger

2.478564119115134

In [79]:
plot_result(data_bollinger)

IndexError: invalid index to scalar variable.